<a href="https://colab.research.google.com/github/chenliyo99/pyspark/blob/main/FaceAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detect Faces

In [ ]:
subscription_key = 'e678c85180174dc7a7e1f457949dddde'

In [ ]:
import urllib.parse, http.client, json

headers = {
    'Ocp-Apim-subscription-Key':subscription_key,
    'Content-Type':'application/json'
}

body = {
    'url':'https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Angela_Merkel_2019_cropped.jpg/1200px-Angela_Merkel_2019_cropped.jpg'
}

params = urllib.parse.urlencode({
    'returnFaceId':True,
    'returnFaceLandmarks':True,
    'returnFaceAttributes':'age,gender,makeup,smile,glasses,accessories,emotion'
})

url = 'southcentralus.api.cognitive.microsoft.com'

endpoint = '/face/v1.0/detect?%s'

try:
  conn = http.client.HTTPSConnection(url)
  conn.request('POST', endpoint % params, str(body), headers)
  response = conn.getresponse()
  jsonData = response.read()
  data = json.loads(jsonData)
  detect_face_id = data[0]['faceId']
  print(json.dumps(data, sort_keys=True, indent=2))
  conn.close()
except Exception as ex:
  print(ex)

[
  {
    "faceAttributes": {
      "accessories": [],
      "age": 57.0,
      "emotion": {
        "anger": 0.0,
        "contempt": 0.0,
        "disgust": 0.0,
        "fear": 0.0,
        "happiness": 0.139,
        "neutral": 0.861,
        "sadness": 0.0,
        "surprise": 0.0
      },
      "gender": "female",
      "glasses": "NoGlasses",
      "makeup": {
        "eyeMakeup": false,
        "lipMakeup": false
      },
      "smile": 0.139
    },
    "faceId": "74c682a5-1128-4944-a699-df0529539aee",
    "faceLandmarks": {
      "eyeLeftBottom": {
        "x": 479.1,
        "y": 549.3
      },
      "eyeLeftInner": {
        "x": 515.1,
        "y": 539.6
      },
      "eyeLeftOuter": {
        "x": 444.0,
        "y": 540.3
      },
      "eyeLeftTop": {
        "x": 476.1,
        "y": 518.8
      },
      "eyeRightBottom": {
        "x": 737.4,
        "y": 566.3
      },
      "eyeRightInner": {
        "x": 702.0,
        "y": 552.1
      },
      "eyeRightOuter": {
  

# Create a Group

In [ ]:
group_id = 'random_people'

group_body = {
    'name':'Random People'
}

try:
  conn = http.client.HTTPSConnection(url)
  conn.request('PUT', '/face/v1.0/persongroups/%s' % group_id, str(group_body), headers)
  response = conn.getresponse()
  data = response.read()
  print(data)
  conn.close()
except Exception as ex:
  print(ex)

b'{"error":{"code":"PersonGroupExists","message":"Person group \'random_people\' already exists."}}'


# Adding person to group

In [ ]:
person_body = {
    'name':'Elizabeth'
}

person_endpoint = '/face/v1.0/persongroups/{0}/persons'.format(group_id)

try:
  conn = http.client.HTTPSConnection(url)
  conn.request('POST', person_endpoint, str(person_body), headers)
  response = conn.getresponse()
  jsonData = response.read()
  data = json.loads(jsonData)
  print(data)
  conn.close()
except Exception as ex:
  print(ex)

{'personId': '29fb53f0-56ad-499d-a9d7-b763b6d938c5'}


In [ ]:
group_id = 'random_people'

endpoint = '/face/v1.0/persongroups/{0}/persons'.format(group_id)

try:
  conn = http.client.HTTPSConnection(url)
  conn.request('GET', endpoint, '', headers)
  response = conn.getresponse()
  jsonData = response.read()
  person_data = json.loads(jsonData)
  print(json.dumps(person_data, sort_keys=True, indent=2))
  conn.close()
except Exception as ex:
  print(ex)

[
  {
    "name": "Elizabeth",
    "persistedFaceIds": [
      "0aa6f776-ce53-4946-ad68-fd961bdd39ce",
      "76eb3bbf-13cd-442d-9f5e-72066ada742e",
      "fe074a09-621d-4dc9-a6f8-b475990734fb"
    ],
    "personId": "29fb53f0-56ad-499d-a9d7-b763b6d938c5"
  },
  {
    "name": "Bill",
    "persistedFaceIds": [
      "1b42ee02-81c6-4cbb-89de-52bf1192e317",
      "4c55c28d-3576-4e94-9a1d-86e4fb3c24d0",
      "9da318d9-7cde-4c00-a1ec-652c6e33d26c"
    ],
    "personId": "c2f66164-4fbd-4a82-bb52-6ac2d92ce47c"
  },
  {
    "name": "Angela",
    "persistedFaceIds": [
      "479e048d-7c02-4a15-9de4-340ee63f047e",
      "a7f41db4-4225-412a-98bd-67350b43f285",
      "fe8e8cf9-b479-4f66-8c81-a5a31cbc897f"
    ],
    "personId": "c47a577a-b74f-41af-9bd5-52863ad92fb0"
  }
]


In [ ]:
try:
  conn = http.client.HTTPSConnection(url)
  conn.request('DELETE', endpoint + '/f465cd5f-55fb-43e3-9852-471a2a11f50d', str(person_body), headers)
  response = conn.getresponse()
  jsonData = response.read()
  print(jsonData)
  conn.close()
except Exception as ex:
  print(ex)

Expecting value: line 1 column 1 (char 0)


# Adding faces to a person

In [ ]:
import os

person_group_id = 'random_people'

url = 'southcentralus.api.cognitive.microsoft.com'

headers = {
    'Ocp-Apim-subscription-Key':subscription_key,
    'Content-Type':'application/octet-stream'
}

for person in person_data:
  face_endpoint = '/face/v1.0/persongroups/{0}/persons/{1}/persistedFaces'.format(group_id, person['personId'])
  for file in os.listdir('people/'+person['name']):
    with open('people/{0}/{1}'.format(person['name'], file), 'rb') as body:
      conn = http.client.HTTPSConnection(url)
      conn.request('POST', face_endpoint, body, headers)
      response = conn.getresponse()
      jsonData = response.read()
      data = json.loads(jsonData)
      print(data)
      conn.close()

{
  "persistedFaceId": "0aa6f776-ce53-4946-ad68-fd961bdd39ce"
}
{
  "persistedFaceId": "76eb3bbf-13cd-442d-9f5e-72066ada742e"
}
{
  "persistedFaceId": "fe074a09-621d-4dc9-a6f8-b475990734fb"
}
{
  "persistedFaceId": "9da318d9-7cde-4c00-a1ec-652c6e33d26c"
}
{
  "persistedFaceId": "1b42ee02-81c6-4cbb-89de-52bf1192e317"
}
{
  "persistedFaceId": "4c55c28d-3576-4e94-9a1d-86e4fb3c24d0"
}
{
  "persistedFaceId": "fe8e8cf9-b479-4f66-8c81-a5a31cbc897f"
}
{
  "persistedFaceId": "a7f41db4-4225-412a-98bd-67350b43f285"
}
{
  "persistedFaceId": "479e048d-7c02-4a15-9de4-340ee63f047e"
}


# Identifying people

In [ ]:
import urllib.parse, http.client, json

url = 'southcentralus.api.cognitive.microsoft.com'

train_headers = {
    'Ocp-Apim-subscription-Key':subscription_key,
}

person_group_id = 'random_people'
train_endpoint = '/face/v1.0/persongroups/{0}/train'.format(person_group_id)

try:
  conn = http.client.HTTPSConnection(url)
  conn.request('POST', train_endpoint, '', train_headers)
  response = conn.getresponse()
  data = response.read()
  print(data)
  conn.close()
except Exception as ex:
  print(ex)

b''


In [ ]:
person_group_id = 'random_people'

identify_endpoint = '/face/v1.0/identify'

identify_headers = {
    'Ocp-Apim-subscription-Key':subscription_key,
}

identify_body = {
    'faceIds':[detect_face_id],
    'personGroupId':person_group_id
}

try:
  conn = http.client.HTTPSConnection(url)
  conn.request('POST', identify_endpoint, str(identify_body), identify_headers)
  response = conn.getresponse()
  jsonData = response.read()
  identify_data = json.loads(jsonData)
  print(identify_data)
  conn.close()
except Exception as ex:
  print(ex)

[{'faceId': '74c682a5-1128-4944-a699-df0529539aee', 'candidates': [{'personId': 'c47a577a-b74f-41af-9bd5-52863ad92fb0', 'confidence': 0.8039}]}]


In [ ]:
print(person_data)

[{'personId': '29fb53f0-56ad-499d-a9d7-b763b6d938c5', 'persistedFaceIds': ['0aa6f776-ce53-4946-ad68-fd961bdd39ce', '76eb3bbf-13cd-442d-9f5e-72066ada742e', 'fe074a09-621d-4dc9-a6f8-b475990734fb'], 'name': 'Elizabeth'}, {'personId': 'c2f66164-4fbd-4a82-bb52-6ac2d92ce47c', 'persistedFaceIds': ['1b42ee02-81c6-4cbb-89de-52bf1192e317', '4c55c28d-3576-4e94-9a1d-86e4fb3c24d0', '9da318d9-7cde-4c00-a1ec-652c6e33d26c'], 'name': 'Bill'}, {'personId': 'c47a577a-b74f-41af-9bd5-52863ad92fb0', 'persistedFaceIds': ['479e048d-7c02-4a15-9de4-340ee63f047e', 'a7f41db4-4225-412a-98bd-67350b43f285', 'fe8e8cf9-b479-4f66-8c81-a5a31cbc897f'], 'name': 'Angela'}]


In [ ]:
found = False
identified_person = identify_data[0]
for candidate in identified_person['candidates']:
  for person in person_data:
    if(person['personId'] == candidate['personId']):
      print('FOUND IT')
      print('This is ' + person['name'])
      found = True

if(found == False):
  print("We couldn't find this person")

FOUND IT
This is Angela
